In [15]:
#%gui qt
from mayavi import mlab
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mimage
%matplotlib inline
import seaborn as sns
from scipy.signal import argrelmax, argrelmin

In [20]:
Nx = 200
Ny = 200
xmin = -0.3
xmax =  1.4
ymin = -1.1
ymax =  1.2

x = np.linspace(xmin, xmax, Nx)
y = np.linspace(ymin, ymax, Ny)
z = np.empty((Nx,Ny))

case = 'globalmin'
#globalmin, nonstationary, saddle


def f(x, y):
    z = y**4 + 0.3*y**3 - 1.5*y**2 - 1.5*x**2 +0.5*x**4
    #if x < 0:
    #    z+= 5*x**2
    return z

def find_nearest(array,value):
    idx = (np.abs(array-value)).argmin()
    return idx, array[idx]
            
for ix, xval in enumerate(x):
    for iy, yval in enumerate(y):
        z[ix,iy] =  f(xval, yval)

zmin = np.amin(z)
zmax = np.amax(z)
origin = (0.5*(xmax-xmin), 0.5*(ymax-ymin), 0.5*(zmax-zmin))
        
if case == 'globalmin':
    xidx, yidx = np.where(np.isclose(z, np.amin(z)))[0]
    xval = x[xidx]
if case == 'saddle':
    xval = 0.0
if case == 'nonstationary':
    xval = 0.7

idxx, nearestx = find_nearest(x, xval)
idxmax = argrelmax(z[idxx, :])[0]
idxmin = argrelmin(z[idxx, :])[0]
if len(idxmax) > 0:
    if len(idxmin > 0):
        for i in idxmax:
            idxmin = np.append(idxmin, i)
        idxy = idxmin
    else:
        idxy = idxmax
else:
    idxy = idxmin
ptx = [nearestx for i in idxy]
pty = [y[i] for i in idxy]
ptz = [f(ptx[i], pty[i])+0.05 for i in range(len(ptx))]
        
x2= np.ones(Nx) * xval
zplane = np.linspace(1.2*np.amin(z), 1.0*np.amax(z), Ny) + 0.2
z2 = np.asarray([zplane for i in range(len(x2))])
cmap = sns.cubehelix_palette(256, start=0.5, rot=-0.75)
cmap = np.asarray([i for i in cmap])
lut = np.ones((256, 4), dtype=np.uint8)*255
lut[:, :3] = np.round(255*cmap).astype(np.uint8)


fig = mlab.figure(size=(1200,1200))
s1 = mlab.surf(x, y, z, colormap='spectral')
plane_col = sns.color_palette('Set2', 10)
s2 = mlab.surf(x2, y, z2.T, color=plane_col[-5], opacity=0.3)
points = mlab.points3d(ptx, pty, ptz, resolution=24, scale_factor=0.1, color=plane_col[-5])
s1.module_manager.scalar_lut_manager.lut.table = lut
mlab.view(azimuth=-45, elevation=50, focalpoint='auto', distance='auto')
imgmap = mlab.screenshot(figure=fig, mode='rgba', antialiased=True)
mlab.savefig('pes.jpeg')
mlab.close(fig)

In [3]:
print z[xidx, yidx]

-1.37903210303
